## Vyrobení Wrapperu okolo C funkce pomocí Cythonu
K načtení C filu z Cythonu budeme potřebovat následující příkazy:

- `cdef extern from "matrix_vector_multiply.c"`: Tento příkaz importuje externí C funkci `matrix_vector_multiply` z hlavičkového souboru "matrix_vector_multiply.c". Tímto způsobem lze v Cythonu volat funkce napsané v jazyce C.
ozhraní mezi rychlými C funkcemi a Python kódem.
- `cnp.ndarray[cnp.float32_t, ndim=2]`, `cnp.ndarray[cnp.float32_t, ndim=1]`: Tyto typy definují NumPy pole s konkrétním datovým typem a počtem dimenzí. V tomto případě se jedná o pole s jednoduchou přesností plovoucích čísel (float32) a jednorozměrné nebo dvourozměrné pole.
- `<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0]`: Tyto konstrukce provádějí type casting ukazatelů na datový typ `float*`. Díky tomu je možné předat adresy prvků NumPy polí do C funkce.

Pro kompilaci je třeba dodat cestu odkud se mají brát C soubory. Při použití cell magic stačí přidat `-I .` parametr, ten řekne cythonu, že se má koukat na zdrojáky do současné složky.

In [ ]:
import numpy as np

matrix = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

In [ ]:
%load_ext Cython

In [ ]:
%%cython -I .
cimport numpy as cnp
import numpy as np

cdef extern from "matrix_vector_multiply.c":
    void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols)

def mvp_func_cy(cnp.ndarray[cnp.float32_t, ndim=2] matrix, cnp.ndarray[cnp.float32_t, ndim=1] vector, cnp.ndarray[cnp.float32_t, ndim=1] result):
    cdef int rows = matrix.shape[0]
    cdef int cols = matrix.shape[1]
    
    matrix_vector_multiply(<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0], rows, cols)


In [ ]:
mvp_func_cy(matrix, vector, result)

print("Result:")
print(result)

Pokud bychom chtěli použít setup.py, bude to vypadat následovně:

In [ ]:
%%writefile mvp_cy.pyx

cimport numpy as cnp
import numpy as np

cdef extern from "matrix_vector_multiply.c":
    void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols)

def mvp_func_cy(cnp.ndarray[cnp.float32_t, ndim=2] matrix, cnp.ndarray[cnp.float32_t, ndim=1] vector, cnp.ndarray[cnp.float32_t, ndim=1] result):
    cdef int rows = matrix.shape[0]
    cdef int cols = matrix.shape[1]
    
    matrix_vector_multiply(<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0], rows, cols)


In [ ]:
%%writefile setup.py

from setuptools import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext_modules = [
    Extension(
        "mvp_cy",
        ["mvp_cy.pyx"],
        include_dirs=[np.get_include(),"."],
        language="c",
    )
]

setup(
    name="mvp_cy",
    ext_modules=cythonize(ext_modules),
)


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import mvp_cy

In [ ]:
mvp_cy.mvp_func_cy(matrix, vector, result)
print("Result:")
print(result)